In [2]:
%load_ext autoreload
%autoreload 2
%pylab inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import open_spiel.python.examples.ubc_dispatch as dispatch
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [142]:
overrides = f'--br_portfolio_path feb22 --eval_every 1_000_000 --num_training_episodes 10_000_000 --br_overrides "--num_training_episodes 1_000_000"'
dispatch.dispatch_experiments('feb22', base_job_name='infocompare', game_name='large_game_2', overrides = overrides, n_seeds=10)
dispatch.dispatch_experiments('feb22', base_job_name='infocompare', game_name='large_game_2_hide', overrides = overrides, n_seeds=10)

Submitted batch job 2490
Submitted batch job 2491
Submitted batch job 2492
Submitted batch job 2493
Submitted batch job 2494
Submitted batch job 2495
Submitted batch job 2496
Submitted batch job 2497
Submitted batch job 2498
Submitted batch job 2499
Submitted batch job 2500
Submitted batch job 2501
Submitted batch job 2502
Submitted batch job 2503
Submitted batch job 2504
Submitted batch job 2505
Submitted batch job 2506
Submitted batch job 2507
Submitted batch job 2508
Submitted batch job 2509


In [19]:
# !squeue --format="%.18i %.65j %.2t %.10M %.6D %R" | grep "_mlp  R" | awk '{print $1}' | xargs scancel

In [160]:
!squeue --format="%.18i %.65j %.2t %.10M %.6D %R"

             JOBID                                                              NAME ST       TIME  NODES NODELIST(REASON)
              2490                             large_game_2-feb22mlp-100_infocompare  R   23:29:33      1 ip-10-255-7-45
              3948     br_1_infocompare_large_game_2-feb22mlp-103_10000000_feb22_mlp  R    1:12:02      1 ip-10-255-7-115


In [79]:
# !scancel 5306 5307

In [110]:
!scancel -u ubuntu

In [112]:
!rm -rf /shared/outputs/minitest/
try:
    Experiment.objects.get(name='minitest').delete()
except:
    pass
dispatch.dispatch_experiments('feb2', base_job_name='minitest', game_name='tiny', overrides = '--br_portfolio_path feb2 --eval_every 500 --num_training_episodes 1_000 --br_overrides "--num_training_episodes 1_000" --eval_overrides "--num_samples 100"')

Submitted batch job 2422
Submitted batch job 2423
